In [1]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization

In [2]:
img_rows, img_cols = 224, 224

In [3]:
vgg = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

In [4]:
for layer in vgg.layers:
    layer.trainable = False

In [5]:
for i,layer in enumerate(vgg.layers):
    print(i,layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [6]:
def lw(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

In [7]:
num_classes = 5
FC_Head = lw(vgg, num_classes)
model = Model(inputs=vgg.input, outputs=FC_Head)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'train_images'

train_data_gen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=45,
                                    width_shift_range=0.3,
                                    height_shift_range=0.3,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [10]:
batch_size = 32

train_generator = train_data_gen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 8 images belonging to 5 classes.


In [11]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
checkpoint = ModelCheckpoint(
    'face recognition.h5',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [13]:
callbacks = [earlystop, checkpoint]
model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(lr=0.001),
    metrics=['accuracy']
)

c:\Users\saivishal radham\Desktop\Stuff\python\AI\face_recog\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
! pip install scipy

In [16]:
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = epochs,
)

C:\Users\saivishal radham\AppData\Local\Temp\ipykernel_28952\2550083477.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


NameError: name 'ndimage' is not defined

In [19]:
from keras.models import load_model

classifier = load_model('face recognisation.h5')

import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join


In [ ]:
image = cv2.imread('images/1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image = image.astype("float32")
image = np.expand_dims(image, axis=0)
pred = 